In [1]:
import sys; sys.path.append("../src")
from direct import Direct
import numpy as np, matplotlib.pyplot as plt

In [2]:
# initial and final states
x0 = [0,0,0,0]
xf = [1,0,0,0]

In [3]:
# instantiate direct segment
seg = Direct(x0, xf)

In [6]:
# optimise trajectory
dv = seg.solve(20, Tub=10, obj='time')

In [7]:
dv

array([ 4.58832435e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.00000000e+00, -2.91588925e-02, -2.41490755e-01,
        2.93000859e-02,  2.43821943e-01, -1.00000000e+00, -1.10600266e-01,
       -4.08005921e-01,  1.12829691e-01,  4.26241984e-01, -3.79060370e-01,
       -2.06778947e-01, -3.33030331e-01,  2.17784658e-01,  3.92691025e-01,
        9.99999996e-01, -2.58043801e-01, -9.15395761e-02,  2.91432647e-01,
        2.20867980e-01,  1.00000000e+00, -2.50990870e-01,  1.49951180e-01,
        3.25751170e-01,  6.50897227e-02,  1.00000000e+00, -1.85620153e-01,
        3.91441936e-01,  3.23257356e-01, -8.58703554e-02,  1.00000000e+00,
       -6.19316522e-02,  6.32932691e-01,  2.83766713e-01, -2.43178524e-01,
        9.99999998e-01,  1.20074634e-01,  8.74423445e-01,  2.04390907e-01,
       -4.18074315e-01,  9.99999991e-01,  3.60398704e-01,  1.11591419e+00,
        7.95274282e-02, -6.21713691e-01,  9.99999955e-01,  6.39601296e-01,
        1.11591419e+00, -